In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import numpy as np

In [2]:
class RNN2(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)



In [3]:
def generate_seq(batch_size):
    input = []
    target = []
    T = 50
    for j in range(batch_size):
        frequency = (5 - 1) * torch.rand(1, 1) + 1
        theta = (5 - 1) * torch.rand(1, 1) + 1
        amplitude = (5 - 1) * torch.rand(1, 1) + 1
        time = torch.linspace(0, 5, steps=T)
        x_p = torch.cat((frequency,theta,amplitude),0)
        x = x_p.tile((T))
        y = amplitude * torch.sin(frequency * time + theta)
        input.append(x)
        target.append(y)

    input_seq = torch.transpose(torch.stack(input, dim=0), 1, 2)
    target_seq = torch.stack(target, dim=0).view(batch_size,T,1)
    
    return(input_seq,target_seq)

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size):
        super(RNN, self).__init__()

        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(input_size, hidden_dim, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.hidden_dim)
        return hidden

In [5]:
cfd = RNN(input_size=3, output_size=1, hidden_dim=100)
n_epochs = 50000
lr=0.01
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(cfd.parameters(), lr=lr)

In [6]:
for epoch in range(1, n_epochs + 1):
    input_seq,target_seq = generate_seq(batch_size = 100)

    optimizer.zero_grad() 

    output, hidden = cfd(input_seq)
    
    loss = criterion(output, target_seq.view(-1,1))
    
    loss.backward() 
    optimizer.step() 
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/50000............. Loss: 5.5352
Epoch: 20/50000............. Loss: 5.2667
Epoch: 30/50000............. Loss: 5.0930
Epoch: 40/50000............. Loss: 5.0550
Epoch: 50/50000............. Loss: 5.3820
Epoch: 60/50000............. Loss: 5.0342
Epoch: 70/50000............. Loss: 5.9425
Epoch: 80/50000............. Loss: 4.9474
Epoch: 90/50000............. Loss: 5.7296
Epoch: 100/50000............. Loss: 4.6186
Epoch: 110/50000............. Loss: 4.7884
Epoch: 120/50000............. Loss: 5.2799
Epoch: 130/50000............. Loss: 4.8809
Epoch: 140/50000............. Loss: 4.4977
Epoch: 150/50000............. Loss: 4.9557
Epoch: 160/50000............. Loss: 4.3543
Epoch: 170/50000............. Loss: 4.5078
Epoch: 180/50000............. Loss: 4.7085
Epoch: 190/50000............. Loss: 5.3272
Epoch: 200/50000............. Loss: 4.6463
Epoch: 210/50000............. Loss: 4.9015
Epoch: 220/50000............. Loss: 4.5229
Epoch: 230/50000............. Loss: 4.7914
Epoch: 240/50000....

Epoch: 1900/50000............. Loss: 4.4581
Epoch: 1910/50000............. Loss: 4.4734
Epoch: 1920/50000............. Loss: 5.2636
Epoch: 1930/50000............. Loss: 4.4527
Epoch: 1940/50000............. Loss: 4.3426
Epoch: 1950/50000............. Loss: 4.8891
Epoch: 1960/50000............. Loss: 4.3330
Epoch: 1970/50000............. Loss: 4.5989
Epoch: 1980/50000............. Loss: 5.0998
Epoch: 1990/50000............. Loss: 4.7856
Epoch: 2000/50000............. Loss: 4.9273
Epoch: 2010/50000............. Loss: 4.8631
Epoch: 2020/50000............. Loss: 4.4864
Epoch: 2030/50000............. Loss: 5.3206
Epoch: 2040/50000............. Loss: 4.2793
Epoch: 2050/50000............. Loss: 4.8333
Epoch: 2060/50000............. Loss: 4.3031
Epoch: 2070/50000............. Loss: 4.6731
Epoch: 2080/50000............. Loss: 5.5900
Epoch: 2090/50000............. Loss: 4.8309
Epoch: 2100/50000............. Loss: 4.3333
Epoch: 2110/50000............. Loss: 5.7674
Epoch: 2120/50000............. L

Epoch: 3770/50000............. Loss: 4.3908
Epoch: 3780/50000............. Loss: 4.9668
Epoch: 3790/50000............. Loss: 4.5746
Epoch: 3800/50000............. Loss: 5.5509
Epoch: 3810/50000............. Loss: 4.9088
Epoch: 3820/50000............. Loss: 4.9544
Epoch: 3830/50000............. Loss: 4.6973
Epoch: 3840/50000............. Loss: 5.3798
Epoch: 3850/50000............. Loss: 4.6801
Epoch: 3860/50000............. Loss: 4.9429
Epoch: 3870/50000............. Loss: 4.9456
Epoch: 3880/50000............. Loss: 4.5729
Epoch: 3890/50000............. Loss: 4.9270
Epoch: 3900/50000............. Loss: 4.7041
Epoch: 3910/50000............. Loss: 4.6373
Epoch: 3920/50000............. Loss: 4.4639
Epoch: 3930/50000............. Loss: 4.9728
Epoch: 3940/50000............. Loss: 5.0059
Epoch: 3950/50000............. Loss: 4.4604
Epoch: 3960/50000............. Loss: 5.3964
Epoch: 3970/50000............. Loss: 4.4010
Epoch: 3980/50000............. Loss: 5.2511
Epoch: 3990/50000............. L

Epoch: 5640/50000............. Loss: 4.6455
Epoch: 5650/50000............. Loss: 4.4180
Epoch: 5660/50000............. Loss: 4.0461
Epoch: 5670/50000............. Loss: 4.3772
Epoch: 5680/50000............. Loss: 4.5164
Epoch: 5690/50000............. Loss: 4.6422
Epoch: 5700/50000............. Loss: 4.5615
Epoch: 5710/50000............. Loss: 4.7204
Epoch: 5720/50000............. Loss: 5.2233
Epoch: 5730/50000............. Loss: 5.0176
Epoch: 5740/50000............. Loss: 4.7405
Epoch: 5750/50000............. Loss: 5.5737
Epoch: 5760/50000............. Loss: 4.7625
Epoch: 5770/50000............. Loss: 5.4520
Epoch: 5780/50000............. Loss: 4.8440
Epoch: 5790/50000............. Loss: 4.9010
Epoch: 5800/50000............. Loss: 4.4938
Epoch: 5810/50000............. Loss: 4.3209
Epoch: 5820/50000............. Loss: 4.2755
Epoch: 5830/50000............. Loss: 4.8672
Epoch: 5840/50000............. Loss: 4.6194
Epoch: 5850/50000............. Loss: 4.6887
Epoch: 5860/50000............. L

Epoch: 7510/50000............. Loss: 5.2472
Epoch: 7520/50000............. Loss: 4.5683
Epoch: 7530/50000............. Loss: 4.8502
Epoch: 7540/50000............. Loss: 4.6438
Epoch: 7550/50000............. Loss: 5.0057
Epoch: 7560/50000............. Loss: 4.6938
Epoch: 7570/50000............. Loss: 5.0588
Epoch: 7580/50000............. Loss: 4.8646
Epoch: 7590/50000............. Loss: 4.4079
Epoch: 7600/50000............. Loss: 4.4133
Epoch: 7610/50000............. Loss: 4.8948
Epoch: 7620/50000............. Loss: 4.5968
Epoch: 7630/50000............. Loss: 4.3706
Epoch: 7640/50000............. Loss: 5.5455
Epoch: 7650/50000............. Loss: 5.5224
Epoch: 7660/50000............. Loss: 4.7589
Epoch: 7670/50000............. Loss: 5.0781
Epoch: 7680/50000............. Loss: 4.4008
Epoch: 7690/50000............. Loss: 4.8862
Epoch: 7700/50000............. Loss: 4.6741
Epoch: 7710/50000............. Loss: 4.5640
Epoch: 7720/50000............. Loss: 4.8039
Epoch: 7730/50000............. L

...

Epoch: 42330/50000............. Loss: 4.2615
Epoch: 42340/50000............. Loss: 4.6833
Epoch: 42350/50000............. Loss: 4.8136
Epoch: 42360/50000............. Loss: 4.5483
Epoch: 42370/50000............. Loss: 5.2770
Epoch: 42380/50000............. Loss: 5.3169
Epoch: 42390/50000............. Loss: 4.4670
Epoch: 42400/50000............. Loss: 4.7739
Epoch: 42410/50000............. Loss: 4.2293
Epoch: 42420/50000............. Loss: 5.0265
Epoch: 42430/50000............. Loss: 4.8420
Epoch: 42440/50000............. Loss: 4.6732
Epoch: 42450/50000............. Loss: 4.9357
Epoch: 42460/50000............. Loss: 4.8655
Epoch: 42470/50000............. Loss: 4.4066
Epoch: 42480/50000............. Loss: 4.9350
Epoch: 42490/50000............. Loss: 4.2288
Epoch: 42500/50000............. Loss: 4.9219
Epoch: 42510/50000............. Loss: 4.6214
Epoch: 42520/50000............. Loss: 5.0346
Epoch: 42530/50000............. Loss: 5.1661
Epoch: 42540/50000............. Loss: 5.2597
Epoch: 425

Epoch: 44160/50000............. Loss: 4.7774
Epoch: 44170/50000............. Loss: 4.6820
Epoch: 44180/50000............. Loss: 4.7754
Epoch: 44190/50000............. Loss: 4.8983
Epoch: 44200/50000............. Loss: 4.3534
Epoch: 44210/50000............. Loss: 4.3834
Epoch: 44220/50000............. Loss: 4.7541
Epoch: 44230/50000............. Loss: 4.5545
Epoch: 44240/50000............. Loss: 4.8315
Epoch: 44250/50000............. Loss: 4.7635
Epoch: 44260/50000............. Loss: 5.2314
Epoch: 44270/50000............. Loss: 4.6172
Epoch: 44280/50000............. Loss: 4.6529
Epoch: 44290/50000............. Loss: 4.7810
Epoch: 44300/50000............. Loss: 4.1719
Epoch: 44310/50000............. Loss: 4.9284
Epoch: 44320/50000............. Loss: 4.6194
Epoch: 44330/50000............. Loss: 4.2679
Epoch: 44340/50000............. Loss: 5.2127
Epoch: 44350/50000............. Loss: 5.0038
Epoch: 44360/50000............. Loss: 4.2856
Epoch: 44370/50000............. Loss: 4.5341
Epoch: 443

Epoch: 45990/50000............. Loss: 4.6472
Epoch: 46000/50000............. Loss: 4.8113
Epoch: 46010/50000............. Loss: 5.0159
Epoch: 46020/50000............. Loss: 4.6799
Epoch: 46030/50000............. Loss: 4.5263
Epoch: 46040/50000............. Loss: 4.3833
Epoch: 46050/50000............. Loss: 5.1102
Epoch: 46060/50000............. Loss: 4.2976
Epoch: 46070/50000............. Loss: 4.6554
Epoch: 46080/50000............. Loss: 4.6516
Epoch: 46090/50000............. Loss: 4.4361
Epoch: 46100/50000............. Loss: 4.4240
Epoch: 46110/50000............. Loss: 4.5594
Epoch: 46120/50000............. Loss: 5.1819
Epoch: 46130/50000............. Loss: 4.6946
Epoch: 46140/50000............. Loss: 5.0152
Epoch: 46150/50000............. Loss: 4.8750
Epoch: 46160/50000............. Loss: 4.8311
Epoch: 46170/50000............. Loss: 4.5173
Epoch: 46180/50000............. Loss: 4.5685
Epoch: 46190/50000............. Loss: 5.4913
Epoch: 46200/50000............. Loss: 4.7151
Epoch: 462

Epoch: 47820/50000............. Loss: 5.1484
Epoch: 47830/50000............. Loss: 4.9547
Epoch: 47840/50000............. Loss: 4.7753
Epoch: 47850/50000............. Loss: 5.2948
Epoch: 47860/50000............. Loss: 4.7403
Epoch: 47870/50000............. Loss: 4.9052
Epoch: 47880/50000............. Loss: 5.1241
Epoch: 47890/50000............. Loss: 4.8501
Epoch: 47900/50000............. Loss: 4.3972
Epoch: 47910/50000............. Loss: 4.3736
Epoch: 47920/50000............. Loss: 4.8044
Epoch: 47930/50000............. Loss: 4.5331
Epoch: 47940/50000............. Loss: 4.6208
Epoch: 47950/50000............. Loss: 4.7894
Epoch: 47960/50000............. Loss: 5.0422
Epoch: 47970/50000............. Loss: 4.4986
Epoch: 47980/50000............. Loss: 4.9441
Epoch: 47990/50000............. Loss: 5.0058
Epoch: 48000/50000............. Loss: 5.0618
Epoch: 48010/50000............. Loss: 4.5894
Epoch: 48020/50000............. Loss: 4.6631
Epoch: 48030/50000............. Loss: 5.0910
Epoch: 480

Epoch: 49650/50000............. Loss: 4.8424
Epoch: 49660/50000............. Loss: 5.2845
Epoch: 49670/50000............. Loss: 4.9523
Epoch: 49680/50000............. Loss: 4.9412
Epoch: 49690/50000............. Loss: 4.6064
Epoch: 49700/50000............. Loss: 4.9213
Epoch: 49710/50000............. Loss: 5.0106
Epoch: 49720/50000............. Loss: 4.8616
Epoch: 49730/50000............. Loss: 4.9097
Epoch: 49740/50000............. Loss: 4.3665
Epoch: 49750/50000............. Loss: 5.0086
Epoch: 49760/50000............. Loss: 4.7105
Epoch: 49770/50000............. Loss: 5.1671
Epoch: 49780/50000............. Loss: 4.6560
Epoch: 49790/50000............. Loss: 4.2891
Epoch: 49800/50000............. Loss: 4.8845
Epoch: 49810/50000............. Loss: 5.4519
Epoch: 49820/50000............. Loss: 4.7045
Epoch: 49830/50000............. Loss: 4.5464
Epoch: 49840/50000............. Loss: 5.1794
Epoch: 49850/50000............. Loss: 4.7097
Epoch: 49860/50000............. Loss: 4.8291
Epoch: 498